In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from unidecode import unidecode

In [ ]:
df_15 = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\cardp_15_19_c_index\tabulação_cardápios_com_index - Tabela_2015.csv')
df_16 = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\cardp_15_19_c_index\tabulação_cardápios_com_index - Tabela_2016.csv')
df_17 = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\cardp_15_19_c_index\tabulação_cardápios_com_index - Tabela_2017.csv')
df_18 = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\cardp_15_19_c_index\tabulação_cardápios_com_index - Tabela_2018.csv')
df_19 = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\cardp_15_19_c_index\tabulação_cardápios_com_index - Tabela_2019.csv')
df_refeicao = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\alimentos - alimentos.csv')

In [ ]:
df_refeicao = df_refeicao[['index','refeição']]
df_refeicao['ali'] = df_refeicao['index'].str.lower().str.replace(" ","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s)

In [ ]:
dfs = [df_15,df_16,df_17,df_18,df_19]

In [ ]:
pnl = []
j = 2015
for df in dfs:
    df[['tcdp','cagr','ali']] = df[['Tipo_cardápio','Cidades_agrupamento_corrigida','Alim1']].apply(lambda x:x.str.lower().str.replace(" ","").str.replace(",","").str.replace(".","").str.replace("-","").str.replace(":","").apply(lambda s:unidecode(s) if type(s) == str else s))
    df = pd.merge(df,df_refeicao,how='left',left_on=['ali'],right_on=['ali'])
    df['Cozinheiros_educacao'] = 0
    df.loc[(df['Tipo_cardápio'].str.lower().str.contains('cozinheiros') == True),'Cozinheiros_educacao'] = 1    
    df.drop(columns=['Agrupamento','ali','tcdp','cagr','index'],inplace = True)
    df['ano'] = j
    pnl.append(df)
    j += 1

In [ ]:
df = pd.concat(pnl)

In [ ]:
df.loc[(df['Cozinheiros_educacao'] > 0),'Cozinheiros_educacao'] = 1

In [ ]:
#Passando para a coluna cidades a lista com os nomes de todas as cidades do agrupamento
df['cidades'] = df['Cidades_agrupamento_corrigida'].str.lower().str.replace(' ','').str.replace('.','').apply(lambda s:unidecode(s) if type(s) == str else s).str.split(',')

In [ ]:
#Expandindo a coluna cidades para cada cidade da lista ser uma linha
df = df.explode('cidades',ignore_index = True)

In [ ]:
df_cidades = pd.read_csv(r'C:\Users\guilh\Documents\Pesquisa\df_cidades_vf_corrigida_21_06 - Sheet1.csv',usecols=['Unnamed: 0','ID Cidade','Nome_cidade'])

In [ ]:
#Merge para termos o nome correto da cidade
df_vf = pd.merge(df,df_cidades,how='left',left_on=['cidades'],right_on = ['Unnamed: 0'])

In [ ]:
df_vf.set_index(['Nome_cidade','ano']).sort_values(by=['Nome_cidade','ano']).to_csv(r'painel_cardapios_com_index_por_cidade.csv')